In [5]:
import os
import pandas as pd
from pathlib import Path
from pandas import DataFrame

In [6]:
%reload_ext dotenv
%dotenv

In [7]:
BASE_PATH               = os.getenv('BASE_PATH')
INDICE_NAME             = os.getenv('INDICE_NAME')
ENGINE_NAME             = os.getenv('ENGINE_NAME')
NOME_COLUNA_DATA        = os.getenv('COLUNA_DATA')
NOME_COLUNA_SALDO       = os.getenv('COLUNA_SALDO')
NOME_INSTITUICAO        = os.getenv('NOME_INSTITUICAO')
NOME_COLUNA_LANCAMENTO  = os.getenv('COLUNA_LANCAMENTO')
DECIMAL_INDICATOR       = os.getenv('DECIMAL_INDICATOR')
NOME_COLUNA_INSTITUICAO = os.getenv('COLUNA_INSTITUICAO')
FILE_TYPE_TO_IMPORT     = os.getenv('FILE_TYPE_TO_IMPORT')
FILE_TYPE_TO_EXPORT     = os.getenv('FILE_TYPE_TO_EXPORT')
IMPORTED_FILE_SEPARATOR = os.getenv('IMPORTED_FILE_SEPARATOR')
VALUES_TO_IGNORE        = os.getenv('VALUES_TO_IGNORE').split(",")

In [8]:
def convert_dataframe_to_notion_format(
        df: DataFrame,
        nome_instituicao: str,
        coluna_data: str,
        coluna_saldo: str,
        coluna_instituicao: str,
        file_uses_day_first = True
    ) -> DataFrame:
    df[coluna_instituicao] = nome_instituicao
    df[coluna_saldo] = pd.to_numeric(df[coluna_saldo])
    df[coluna_data] = pd.to_datetime(df[coluna_data], dayfirst=file_uses_day_first).dt.strftime('%m/%d/%Y')
    return df

In [9]:
def filter_dataframe(
        df: DataFrame,
        coluna_lancamento: str,
        values_to_ignore
    ) -> DataFrame:
    return df[~df[coluna_lancamento].isin(values_to_ignore)]

In [10]:
def get_file_name_to_save(
        df: DataFrame,
        coluna: str,
        indice: int,
        type: str
    ) -> str:
    value = df[coluna][indice]
    return f'{value[-4:]}-{value[0:2]}.{type}'

In [11]:
def save_dataframe_to_csv(
        df: DataFrame,
        coluna: str,
        indice: str,
        file_type: str,
        show_index = False
    ):
    filename = get_file_name_to_save(df, coluna, int(indice), file_type)
    df.to_csv(f'{filename}', index=show_index)

In [12]:
for file in os.listdir(BASE_PATH):
    if file.endswith(FILE_TYPE_TO_IMPORT):
        df = pd.read_csv(
            Path(file),
            sep=IMPORTED_FILE_SEPARATOR,
            engine=ENGINE_NAME,
            names=[NOME_COLUNA_DATA, NOME_COLUNA_LANCAMENTO, NOME_COLUNA_SALDO],
            decimal=DECIMAL_INDICATOR
        )

        if df.empty:
            continue
        else:
            df = convert_dataframe_to_notion_format(df, NOME_INSTITUICAO, NOME_COLUNA_DATA, NOME_COLUNA_SALDO, NOME_COLUNA_INSTITUICAO)
            filtered_df = filter_dataframe(df, NOME_COLUNA_LANCAMENTO, VALUES_TO_IGNORE)
            save_dataframe_to_csv(filtered_df, NOME_COLUNA_DATA, INDICE_NAME, FILE_TYPE_TO_EXPORT)